# Deploy the llama2-13b chinese fine tuned model on Amazon SageMaker

As we have finetuned the model, next we will show you how to deploy the model on SageMaker.

In this notebook, we explore how to host a large language model on SageMaker using the [Large Model Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-inference.html) container that is optimized for hosting large models using DJLServing. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent [blog post](https://aws.amazon.com/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/).

## Create a SageMaker Model for Deployment
As a first step, we'll import the relevant libraries and configure several global variables such as the hosting image that will be used nd the S3 location of our model artifacts

In [1]:
import sagemaker
from sagemaker.model import Model
from sagemaker import serializers, deserializers
from sagemaker import image_uris
import boto3
import os
import time
import json
import jinja2
from pathlib import Path

In [2]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3") # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime") # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment() # jinja environment to generate model configuration templates

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


## Build the inference contianer image

In [3]:
%%writefile Dockerfile.inference
## You should change below region code to the region you used, here sample is use us-west-2
#From 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.1-deepspeed0.9.2-cu118
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
#From 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117 

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

## Install transfomers version which support LLaMaTokenizer
#RUN python3 -m pip install git+https://github.com/huggingface/transformers.git@68d640f7c368bcaaaecfc678f11908ebbd3d6176
## Install transfomers version which support vicuna v1.1 LLaMaTokenizer
RUN python3 -m pip install transformers==4.30.2
#RUN python3 -m pip install git+https://github.com/lanking520/DeepSpeed.git@falcon

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

Overwriting Dockerfile.inference


In [4]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-llama2-13b-chinese-inference-demo"

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} -f Dockerfile.inference .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
## The image uri which is build and pushed above
inference_image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account_id, region, repo_name)
inference_image_uri

## Deploying a Large Language Model using Hugging Face Accelerate
The DJL Inference Image which we will be utilizing ships with a number of built-in inference handlers for a wide variety of tasks including:
- `text-generation`
- `question-answering`
- `text-classification`
- `token-classification`

You can refer to this [GitRepo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python) for a list of additional handlers and available NLP Tasks. <br>
These handlers can be utilized as is without having to write any custom inference code. We simply need to create a `serving.properties` text file with our desired hosting options and package it up into a `tar.gz` artifact.

Lets take a look at the `serving.properties` file that we'll be using for our first example

In [91]:
!mkdir llama2-13b-chinese-accelerate_src

mkdir: cannot create directory ‘llama2-13b-chinese-accelerate_src’: File exists


In [ ]:
%%writefile llama2-13b-chinese-accelerate_src/serving.properties
#engine=Python
engine=DeepSpeed
option.entryPoint=model.py
option.tensor_parallel_degree=4
#option.s3url=s3://sagemaker-us-west-2-687912291502/llm/models/LLM_llama2_7b/
option.model_id=TheBloke/Llama-2-13B-fp16
#option.task=text-generation
#option.device_map=auto
option.load_in_8bit=TRUE

There are a few options specified here. Lets go through them in turn<br>
1. `engine` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the [DJL Python Engine](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python)
2. `option.entryPoint` - specifies the entrypoint code that will be used to host the model. djl_python.huggingface refers to the `huggingface.py` module from [djl_python repo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python).  
3. `option.s3url` - specifies the location of the model files. Alternativelly an `option.model_id` option can be used instead to specifiy a model from Hugging Face Hub (e.g. `EleutherAI/gpt-j-6B`) and the model will be automatically downloaded from the Hub. The s3url approach is recommended as it allows you to host the model artifact within your own environment and enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance 
4. `option.task` - This is specific to the `huggingface.py` inference handler and specifies for which task this model will be used
5. `option.device_map` - Enables layer-wise model partitioning through [Hugging Face Accelerate](https://huggingface.co/docs/accelerate/usage_guides/big_modeling#designing-a-device-map). With `option.device_map=auto`, Accelerate will determine where to put each **layer** to maximize the use of your fastest devices (GPUs) and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM). Even if the model is split across several devices, it will run as you would normally expect.
6. `option.load_in_8bit` - Quantizes the model weights to int8 thereby greatly reducing the memory footprint of the model from the initial FP32. See this [blog post](https://huggingface.co/blog/hf-bitsandbytes-integration) from Hugging Face for additional information 

For more information on the available options, please refer to the [SageMaker Large Model Inference Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)

Our initial approach here is to utilize the built-in functionality within Hugging Face Transformers to enable Large Language Model hosting. 

In [176]:
%%writefile llama2-13b-chinese-accelerate_src/requirements.txt
protobuf==3.20
accelerate>=0.21.0
git+https://github.com/huggingface/peft.git
xformers

Overwriting llama2-13b-chinese-accelerate_src/requirements.txt


In [177]:
%%writefile llama2-13b-chinese-accelerate_src/model.py
# -*- coding: utf-8 -*-

from djl_python import Input, Output
import os
import logging
import math
import deepspeed
import torch
import torch.distributed as dist
import sys
import subprocess
import time
import transformers
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers.models.llama.tokenization_llama import LlamaTokenizer
import time
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

print("transformers version=="+transformers.__version__)

predictor = None



def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    #logging.info(f"Loading model in {model_location}")
    
    print("----------tensor parallel is {0}---------".format(tensor_parallel))
    
    #model_location = "EleutherAI/gpt-neo-2.7B"
    logging.info(f"Loading model in {model_location}")
    print(f"Loading model in {model_location}")
    model_name_or_path = model_location
    adapter_name_or_path = 'shareAI/llama2-13b-Chinese-chat'
    save_path = '/tmp/llama2-13b-Chinese-chat_v1'
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name_or_path,
        trust_remote_code=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        #device_map='auto'
    )
    print("load model success")
    model = PeftModel.from_pretrained(model, adapter_name_or_path)
    print("load adapter success")
    model = model.merge_and_unload()
    print("merge success")
    
    ### for deepspeed ####
    model = deepspeed.init_inference(
        model,
        mp_size=tensor_parallel,
        dtype=torch.half,
        replace_method="auto",
        replace_with_kernel_inject=True,
    )

    local_rank = int(os.getenv("LOCAL_RANK", "0"))
    generator = pipeline(
        task="text-generation", model=model, tokenizer=tokenizer, device=local_rank, use_cache=True
    )
    #model.requires_grad_(False)
    #model.eval()
    print("模型加载成功！")
    ### for accelerate ####
    #generator = pipeline(
    #    task="text-generation", model=model, tokenizer=tokenizer, device_map="auto", use_cache=True
    #)
    #generator.tokenizer.pad_token_id = model.config.eos_token_id
    return generator, model, tokenizer


def handle(inputs: Input) -> None:
    global predictor, model, tokenizer
    try:
        if not predictor:
            predictor,model,tokenizer = load_model(inputs.get_properties())

        print(inputs)
        if inputs.is_empty():
            # Model server makes an empty call to warmup the model on startup
            return None
        
        inputs = inputs.get_as_json()
        if not inputs.get("inputs"):
            return Output().add_as_json({"code":-1,"msg":"input field can't be null"})


        #input data
        data = inputs.get("inputs")
        params = inputs.get("parameters",{})
        print(params)
        
        user_input = '{}</s>'.format(data)
        user_input_ids = tokenizer(user_input, return_tensors="pt", add_special_tokens=False).input_ids
        model_input_ids = user_input_ids.to("cuda")
        with torch.no_grad():
          outputs = model.generate(
                input_ids=model_input_ids, max_new_tokens=params["max_new_tokens"], do_sample=False, top_p=params["top_p"],
                temperature=params["temperature"], repetition_penalty=params["repetition_penalty"], eos_token_id=tokenizer.eos_token_id
          )
        model_input_ids_len = model_input_ids.size(1)
        response_ids = outputs[:, model_input_ids_len:]
        response = tokenizer.batch_decode(response_ids)
        print("Bot：" + response[0].strip().replace('</s>', ""))
    except Exception as e:
        return Output().add_as_json({"code":-1,"msg":e})

Overwriting llama2-13b-chinese-accelerate_src/model.py


We place the `serving.properties` file into a tarball and upload it to S3

In [178]:
!tar czvf acc_model.tar.gz llama2-13b-chinese-accelerate_src/ 

llama2-13b-chinese-accelerate_src/
llama2-13b-chinese-accelerate_src/.ipynb_checkpoints/
llama2-13b-chinese-accelerate_src/.ipynb_checkpoints/serving-checkpoint.properties
llama2-13b-chinese-accelerate_src/model.py
llama2-13b-chinese-accelerate_src/serving.properties
llama2-13b-chinese-accelerate_src/requirements.txt


In [179]:
s3_code_prefix = "llama2-13b-chinese/deploy/code"

code_artifact = sess.upload_data("acc_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/llama2-13b-chinese/deploy/code/acc_model.tar.gz


## Deploy Model to a SageMaker Endpoint
With a helper function we can now deploy our endpoint and invoke it with some sample inputs

In [180]:
from sagemaker.utils import name_from_base
import boto3

model_name="llama2-13b-chinese-chat"
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

llama2-13b-chinese-chat
Image going to be used is ---- > 687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-llama2-13b-chinese-inference-demo:latest
Created Model: arn:aws:sagemaker:us-west-2:687912291502:model/llama2-13b-chinese-chat


In [181]:

endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.24xlarge",
            "InitialInstanceCount": 1,
            #"VolumeSizeInGB" : 400,
            "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint-config/llama2-13b-chinese-chat-config',
 'ResponseMetadata': {'RequestId': '74ecec12-5c19-4bf4-bf74-125f9fda4c44',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '74ecec12-5c19-4bf4-bf74-125f9fda4c44',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '111',
   'date': 'Sat, 19 Aug 2023 08:11:48 GMT'},
  'RetryAttempts': 0}}

In [182]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:687912291502:endpoint/llama2-13b-chinese-chat-endpoint


In [183]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:687912291502:endpoint/llama2-13b-chinese-chat-endpoint
Status: InService


Let's run an example with a basic text generation prompt Large model inference is

In [1]:
import boto3
import json
import uuid
prompt_prefix = "你正在一个聊天室里和不同国家的人们聊天，你能读懂所有国家的语言，你负责通过聊天记录分析所有聊天者的性格和有效信息，具体步骤如下：\
1.阅读他们的聊天记录 \
2.总结他们聊天里面的重要信息 \
3.抽象他们的人设 \
4.使用评分体系抽象他们之间的人际关系，然后给一个评分，范围1-10分，分越高关系越好 \
聊天信息如下: " 

chats_infos = """
WaRGazmo : "you lucked out there buddy" 
WarLord : "suerte? eso no existe " 
WarLord : "soy más rápido que la luz " 
WaRGazmo : "it exists.. or karma" 
DirtyE1bow : "so you was a planned birth ?" 
WaRGazmo : "thats what she said bruh" 
WarLord : "te amo mi amor " 
Manowarik : "Мир вам,люди добрые.." 
kotofei : "и тебе боярин, что не подался в челядь королю)" 
XxNORxXMithra : "God morgen folkens :) " 
kotofei : "и прочие жители галактики " 
XxNORxXMithra : "Ja de også forsåvidt :) " 
Manowarik : "Котофей-это который по цепи кругом?Песни там,сказки?😆😆" 
kotofei : "не, то дальний убогий родственник " 
Manowarik : "Эххх..Лукоморье мимо..((" 
kipl : "Котофей он из сказки Лиса и Котофей Иванович. " 
kipl : "Межвидовой брак и крышевание леса" 
kotofei : "лиса 🦊 мералиса и Котофей Иваныч " 
leister : "😆" 
XxFoxyQBAxX : "po co tyle zrobiłeś?"
"""
client = boto3.client('sagemaker-runtime')
endpoint_name = 'llama2-13b-chinese-chat-endpoint'
parameters = {
  "max_length": 2048,
  "temperature": 1,
  "max_new_tokens":500,
  "repetition_penalty": 0.8,
  "top_p":0.8
}

response_model = client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": [prompt_prefix+chats_infos],
                #"inputs": "你好",
                "parameters": parameters,
            }
            ),
            ContentType="application/json",
        )

print(response_model['Body'].read().decode('utf8'))

#payload = '{"inputs" : , \
#            "parameters": { "max_length": 200, "temperature": 0.6 }  \
#           }'
#encoded_inp = (payload).encode("utf-8")
#response = client.invoke_endpoint(
#        EndpointName=endpoint_name,
#        ContentType='application/json',
#        Accept='application/json',
#        Body=payload
#)


ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint llama2-13b-chinese-chat-endpoint of account 687912291502 not found.

In [ ]:
# Clean up the endpoint before proceeding
predictor.delete_endpoint()

## Reference

[sagemaker-hosting/Large-Language-Model-Hosting/](https://github.com/aws-samples/sagemaker-hosting/tree/main/Large-Language-Model-Hosting)